<a href="https://colab.research.google.com/github/MLo7Ghinsan/DiffSinger_colab_notebook_MLo7/blob/main/NSF_hifigan_finetuning_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title # Mount Google Drive and Setup


#ill put the imports here too ig
from IPython.display import clear_output
import os
import shutil
import yaml
from google.colab import drive
drive.mount("/content/drive")
!rm -rf /content/sample_data
!git clone https://github.com/openvpi/DiffSinger.git
!git clone https://github.com/openvpi/SingingVocoders
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install click einops h5py librosa lightning matplotlib mido numpy praat-parselmouth preprocessing pyworld PyYAML torchmetrics tqdm
#for onnx export
!pip install onnx onnxruntime onnxsim
!apt-get install aria2
#this is here bc it wouldnt work with base_task.py so we just use base_task_gan.py for it
shutil.copy("/content/SingingVocoders/training/base_task_gan.py", "/content/SingingVocoders/training/base_task.py")
clear_output()
!aria2c https://github.com/openvpi/SingingVocoders/releases/download/v0.0.1/hifi.7z
!aria2c https://github.com/openvpi/DiffSinger/releases/download/v2.1.0/rmvpe.zip
!7z x /content/hifi.7z -o/content/SingingVocoders/pretrained/hifigan
!7z x /content/rmvpe.zip -o/content/SingingVocoders/pretrained
!rm /content/hifi.7z
!rm /content/rmvpe.zip
clear_output()


In [ ]:
#@title # Extract Data
#@markdown ___

import re
import soundfile as sf
import librosa

#@markdown Path to zip file containing your audio data
data_zip_path = "" # @param {type:"string"}

#markdown Config type you want to use (this one is permanent for now)
#train_config = "hifigan" # @param ["hifigan"]

#@markdown Pitch extractor algorithm
f0_ext = "parselmouth" # @param ["parselmouth", "rmvpe"]

#@markdown Precision option
precision = "16-mixed" # @param ["32-true", "bf16-mixed", "16-mixed"]

#@markdown data aug option
data_aug = True # @param {type:"boolean"}
data_aug_probability = 0.5 # @param {type:"slider", min:0.1, max:3, step:0.1}


#@markdown Model save interval
save_interval = 2000 # @param {type:"slider", min:100, max:10000, step:100}
save_interval = int(save_interval / 2)

#@markdown Amount of validation files you want to use (can't exceed the amount of train files)
val_amount = 8 # @param {type:"slider", min:1, max:18, step:1}


train_path = "/content/audio_data/input"
npz_path = "/content/audio_data/output"


!rm -rf /content/audio_data >/dev/null 2>&1

if not os.path.exists(train_path):
    os.makedirs(train_path)
    os.makedirs(npz_path)
!7z e "$data_zip_path" -o{train_path} *.wav -r

#audio resample to avoid errors
sample_rate = 44100
def resample_and_convert_audio(audio_path, sample_rate):
    audio, sr = librosa.load(audio_path, sr=None)
    if sr != sample_rate:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=sample_rate)
    sf.write(audio_path, audio, sample_rate)
    wav_name = os.path.basename(audio_path)
    print(f"Resample {wav_name} to {sample_rate}")
for root, dirs, files in os.walk(train_path):
    for file in files:
        if file.endswith(".wav"):
            audio_path = os.path.join(root, file)
            resample_and_convert_audio(audio_path, sample_rate)

with open("/content/SingingVocoders/configs/ft_hifigan.yaml", "r") as config:
    ew = yaml.safe_load(config)
ew["data_input_path"] = ["/content/audio_data/input"]
ew["data_out_path"] = ["/content/audio_data/output"]
ew["val_num"] = val_amount
if data_aug:
    ew["key_aug"] = data_aug
    ew["key_aug_prob"] = data_aug_probability
with open("/content/SingingVocoders/configs/ft_hifigan.yaml", "w") as config:
    yaml.dump(ew, config)

with open("/content/SingingVocoders/configs/ft_hifigan.yaml", "r") as config:
    ew = yaml.safe_load(config)
ew["pe"] = f0_ext
ew["val_check_interval"] = save_interval #questionable
ew["pl_trainer_accelerator"] = "gpu"
ew["pl_trainer_precision"] = precision
ew["finetune_ckpt_path"] = "/content/SingingVocoders/pretrained/hifigan/hifi.ckpt"
with open("/content/SingingVocoders/configs/ft_hifigan.yaml", "w") as config:
    yaml.dump(ew, config)
print("\n")

In [ ]:
#@title # Preprocess
#@markdown ___
# edit the f0 min and max for parselmouth
import re
f0_min = 12 #C0
f0_max = 2100 #about C7
wav2F0_path = "/content/SingingVocoders/utils/wav2F0.py"
with open(wav2F0_path, "r") as f:
    change_min_max = f.read()
change_min_max = re.sub(r"f0_min\s*=\s*\d+", f'f0_min = {f0_min}', change_min_max)
change_min_max = re.sub(r"f0_max\s*=\s*\d+", f'f0_max = {f0_max}', change_min_max)
with open(wav2F0_path, "w") as f:
    f.write(change_min_max)

with open("/content/SingingVocoders/process.py", "r") as f:
    process_py_read = f.read()
process_py_read = re.sub(r"val_set\s*=\s*.*", f"val_set = random.choices(list(data_filename_set), k=val_num)", process_py_read)
with open("/content/SingingVocoders/process.py", "w") as f:
    f.write(process_py_read)

%cd /content/SingingVocoders
!python /content/SingingVocoders/process.py --config /content/SingingVocoders/configs/ft_hifigan.yaml --strx 1
%cd /content

In [ ]:
#@title #Tensorboard
#@markdown ___

#@markdown path to your save dir basically
logdir = "" # @param {type:"string"}
%reload_ext tensorboard
%tensorboard --logdir {logdir}

In [ ]:
%cd /content/SingingVocoders
#@title # Training
#@markdown ___

config_path = "/content/SingingVocoders/configs/ft_hifigan.yaml" # @param {type:"string"}
exp_name = "" # @param {type:"string"}
save_path = "" # @param {type:"string"}

import re

training_utils_path = "/content/SingingVocoders/utils/training_utils.py"
with open(training_utils_path, "r") as f:
    edit_relative_path = f.read()
new_relative = "relative_path = filepath.relative_to(Path('/content').resolve())"
pattern = r"relative_path\s*=\s*.*"
edit_relative_path = re.sub(pattern, new_relative, edit_relative_path)
with open(training_utils_path, "w") as f:
    f.write(edit_relative_path)

!python /content/SingingVocoders/train.py --config {config_path} --exp_name {exp_name} --work_dir {save_path}

In [ ]:
#@title # Export ONNX
#@markdown ___
#test exporting because the export_ckpt.py script is you know dying
#this is a must before using diffsinger export script
import torch
import json
import os

#@markdown path to your vocoder ckpt
ckpt_path = "" # @param {type:"string"}
ckpt_folder = os.path.dirname(ckpt_path)
ckpt_config = ckpt_folder + "/config.yaml"

#@markdown your vocoder onnx save path
export_path = "" # @param {type:"string"}
save_path =  export_path + "/model.ckpt"

#@markdown your vocoder name
name = "" # @param {type:"string"}
aaa2x = torch.load(ckpt_path)['state_dict']
ckp = {}
for i in aaa2x:
    i: str
    if 'generator.' in i:
        ckp[i.replace('generator.', '')] = aaa2x[i]
torch.save({'generator': ckp}, save_path)

# copied from openvpi's vocoder
config_data = {
    "resblock": "1",
    "num_gpus": 4,
    "batch_size": 10,
    "learning_rate": 0.0002,
    "adam_b1": 0.8,
    "adam_b2": 0.99,
    "lr_decay": 0.999,
    "seed": 1234,
    "upsample_rates": [8, 8, 2, 2, 2],
    "upsample_kernel_sizes": [16, 16, 4, 4, 4],
    "upsample_initial_channel": 512,
    "resblock_kernel_sizes": [3, 7, 11],
    "resblock_dilation_sizes": [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
    "discriminator_periods": [3, 5, 7, 11, 17, 23, 37],
    "segment_size": 16384,
    "num_mels": 128,
    "num_freq": 1025,
    "n_fft": 2048,
    "hop_size": 512,
    "win_size": 2048,
    "sampling_rate": 44100,
    "fmin": 40,
    "fmax": 16000,
    "fmax_for_loss": None,
    "num_workers": 16,
    "dist_config": {
        "dist_backend": "nccl",
        "dist_url": "tcp://localhost:54321",
        "world_size": 1
    }
}
with open(f"{export_path}/config.json", "w") as json_file:
    json.dump(config_data, json_file, indent=4)

with open(ckpt_config, "r") as config:
    add_vocoder_ckpt = yaml.safe_load(config)
add_vocoder_ckpt["vocoder_ckpt"] = save_path
with open(ckpt_config, "w") as config:
    yaml.dump(add_vocoder_ckpt, config)

!cp {ckpt_config} {export_path}

!python /content/DiffSinger/scripts/export.py nsf-hifigan --config {export_path}/config.yaml --out {export_path} --name {name}